<a href="https://colab.research.google.com/github/luyug/QuCo/blob/main/PromptLibCustomTask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!rm -rf prompt-lib
!git clone https://github.com/reasoning-machines/prompt-lib
!pip install prompt-lib/

Cloning into 'prompt-lib'...
remote: Enumerating objects: 469, done.
remote: Counting objects: 100% (469/469), done.
remote: Compressing objects: 100% (265/265), done.
remote: Total 469 (delta 277), reused 383 (delta 195), pack-reused 0
Receiving objects: 100% (469/469), 2.02 MiB | 14.35 MiB/s, done.
Resolving deltas: 100% (277/277), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./prompt-lib
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for prompt-lib: filename=prompt_lib-0.1-py3-none-any.whl size=42476 sha256=d1c31f2a24b134adedb75f0d663f7d303914d5528dbf2062

**Step 0: Export keys.**

We need to have OPENAI_API_KEY in the environment because we are calling their model, so let's do that first.

In [2]:
import os
os.environ["OPENAI_API_KEY"] = '' #@param {type:"string"}
!export OPENAI_API_KEY

In [3]:
!echo $OPENAI_API_KEY

In [4]:
import json
import wandb
wandb.init(mode="disabled")
# NOTE: prompt-lib has rich wandb logging. We are disabling it just for this notebook

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


**Step 1: Task Configuration** 


`prompt-lib` provides a large number of tasks out of the box, and adding new tasks is trivial. In this notebook, we use a pre-registered task called `gsm_stream`. Here, `gsm` is the name of the task (GSM8k), and `stream` is just a tag that specifies that it's a chain-of-thought task. I saw the `stream` label in Wei et al. repo and have been using it for CoT like tasks for consistency. When we register a new task, we are free to give it any label with some restrictions (see README for details).

In [9]:
task_config_dict = {
    "task_id": "gsm_stream",
    "tag": "gsm_stream",
    "num_prompt_examples": 5,
    "max_tokens": 600,
    "num_questions_per_thread": 100,
    "seed": 0,
    "is_cot_task": True,
    "model_name": "code-davinci-002",
    "max_requests_per_min": 20,
    "cached_timestamp": None,
    "temperature": 0.0,
    "is_debug": True,
    "prompt_config": {
        "question_prefix": "Q: ",
        "answer_prefix": "A: ",
        "final_answer_prefix": "The answer is ",
        "intra_example_sep": "\n",
        "inter_example_sep": "\n\n"
    },
    "eval_function": "get_exact_match_acc"
}

Let's look at some of the important fields fields in detail:



*   `task_id` identifies the task. Crucially, this assumes that you have a file called `data/tasks/gsm.jsonl` with two fields: `input` and `target`. The task file is always extracted from the task name by doing `task_name.split('_')[0]`. 

Also, this assumes that you have `gsm_stream` task registered. See the README for details on how a task is registered. Briefly, task registration requires specifying a name for the task, and the prompt it'll use. 
*   `prompt_config` outlines how the prompt is made. 


Let's add a link to the dataset we need for inference, `gsm.jsonl`. gsm.jsonl is already included in prompt-lib, but not in the current directory. We add it as a soft-link.


In [6]:

!mkdir -p data/tasks
%cd data/tasks
!ln -s ../../prompt-lib/data/tasks/gsm.jsonl gsm.jsonl
%cd ../..

/content/data/tasks
ln: failed to create symbolic link 'gsm.jsonl': File exists
/content


In [7]:
!ls data/tasks


gsm.jsonl


**Step 2: Running the job**

In [10]:
from prompt_lib.prompts.utils import TaskConfig
from prompt_lib.inference import inference_loop
task_config = TaskConfig.from_config_dict(task_config_dict)


In [11]:
outputs = inference_loop(task_config, num_inference_examples=3)

Querying openai [thread_id=0]:  33%|███▎      | 1/3 [00:05<00:11,  5.79s/it]

Janet’s ducks lay 16 eggs per day. She eats 3 for breakfast and bakes 4 for muffins. So she has 16 - 3 - 4 = 9 eggs left. She sells them for $2 each. So she makes 9 * 2 = 18 dollars. The answer is 18



Querying openai [thread_id=0]:  67%|██████▋   | 2/3 [00:08<00:04,  4.86s/it]

A robe takes 2 bolts of blue fiber and half that much white fiber.  Half of 2 is 1.  So it takes 2 + 1 = 3 bolts in total.  The answer is 3



100%|██████████| 1/1 [00:12<00:00, 12.42s/it]

Josh bought the house for 80,000 dollars. He put in 50,000 dollars in repairs. The house increased in value by 150%. So the house is now worth 80,000 + 50,000 + (50,000 * 150%) = 80,000 + 50,000 + 75,000 = 205,000 dollars. He made a profit of 205,000 - 80,000 - 50,000 = 75,000 dollars. The answer is 75,000
Accuracy: 66.67
Accuracy: 66.67


In [13]:
outputs.head(3)

,prompt,question,answer,generated_answer,entire_response,is_correct
0,Q: Olivia has $23. She bought five bagels for ...,\nQ: Janet’s ducks lay 16 eggs per day. She ea...,18,Janet’s ducks lay 16 eggs per day. She eats 3 ...,Janet’s ducks lay 16 eggs per day. She eats 3 ...,1
1,Q: Olivia has $23. She bought five bagels for ...,\nQ: A robe takes 2 bolts of blue fiber and ha...,3,A robe takes 2 bolts of blue fiber and half th...,A robe takes 2 bolts of blue fiber and half th...,1
2,Q: Olivia has $23. She bought five bagels for ...,\nQ: Josh decides to try flipping a house. He...,70000,"Josh bought the house for 80,000 dollars. He p...","Josh bought the house for 80,000 dollars. He p...",0


**That's it! We can use the output for further analysis. A copy of the outputs is also saved at `data/logs/` with necessary hyperparameters in the path**

In [14]:
!ls -R data/logs

data/logs:
gsm_stream

data/logs/gsm_stream:
code-davinci-002

data/logs/gsm_stream/code-davinci-002:
temp_0.0

data/logs/gsm_stream/code-davinci-002/temp_0.0:
seed_0

data/logs/gsm_stream/code-davinci-002/temp_0.0/seed_0:
k_5

data/logs/gsm_stream/code-davinci-002/temp_0.0/seed_0/k_5:
gsm_stream

data/logs/gsm_stream/code-davinci-002/temp_0.0/seed_0/k_5/gsm_stream:
2022-11-30_07-44-30

data/logs/gsm_stream/code-davinci-002/temp_0.0/seed_0/k_5/gsm_stream/2022-11-30_07-44-30:
outputs.jsonl  outputs_part0.jsonl  task_config.json
